In [1]:
import numpy as np
from sklearn.utils import shuffle
from keras.utils import np_utils

terrain_classes = ('Plano', 'DM', 'Elevaciones')

x = np.load('processed_terrain_data.npy')
y = np.load('terrain_data_labels.npy')

x, y = shuffle(x, y)

n_clases = len(terrain_classes)
n_features = len(x[0])

print(np.unique(y, return_counts=True))
print(len(x))
print(len(y))
print(n_clases)
print(n_features)


2021-11-15 14:25:23.729808: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


(array([0, 1, 2]), array([1019,  429,  603]))
2051
2051
3
11


In [2]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                  np.unique(y),
                                                  y)
kf = KFold(n_splits=5, shuffle=True)
acc = 0
recall = np.array([0., 0., 0.])
iteration = 1
for train_index, test_index in kf.split(x):
    print(f'Iteration {iteration}')
    iteration += 1
    x_train = x[train_index, :]
    y_train = y[train_index]
    y_train = np_utils.to_categorical(y_train)

    clf = Sequential()
    clf.add(Dense(32, input_dim=n_features, activation='relu'))
    clf.add(Dense(32, activation='relu'))
    clf.add(Dense(3, activation='softmax'))
    clf.compile(loss='categorical_crossentropy', optimizer='adam')
    history = clf.fit(x_train, y_train, epochs=50, batch_size=5, class_weight=(
        dict(zip(np.unique(y), class_weights))), verbose=0)

    x_test = x[test_index, :]
    y_test = y[test_index]
    y_pred = np.argmax(clf.predict(x_test), axis=-1)
    cm = confusion_matrix(y_test, y_pred)
    acc += (cm[0, 0]+cm[1, 1]+cm[2, 2])/len(y_test)
    recall[0] += cm[0, 0]/(cm[0, 0] + cm[0, 1] + cm[0, 2])
    recall[1] += cm[1, 1]/(cm[1, 0] + cm[1, 1] + cm[1, 2])
    recall[2] += cm[2, 2]/(cm[2, 0] + cm[2, 1] + cm[2, 2])

acc = acc/5
print('ACC = ', acc)
recall = recall/5
print('RECALL = ', recall)

Iteration 1


/home/hivini/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classes=[0 1 2], y=[0 0 2 ... 0 0 2] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
2021-11-15 14:25:24.837419: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-15 14:25:24.845803: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-11-15 14:25:25.122873: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:927] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-15 14:25:25.122941: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 with Max-Q Desig

Iteration 2
Iteration 3
Iteration 4
Iteration 5
ACC =  0.810824283425316
RECALL =  [0.93870618 0.60357072 0.73666083]
